Small experiment testing joblib performance with different setups

In [42]:
import numpy as np
import time

from joblib import Parallel, delayed

from joblib.externals.loky import set_loky_pickler
from joblib import wrap_non_picklable_objects

#! But after tests it seems not to make that big difference
set_loky_pickler('pickle') #allegedly, cloudpickle is extremaly slow for big lists

def timer(prefix):
    def middle(func):
        def wrapper_timer(*args, **kwargs):
            tic = time.perf_counter()
            value = func(*args, **kwargs)
            toc = time.perf_counter()
            elapsed_time = toc - tic
            print(f"{prefix} Elapsed time: {elapsed_time:0.4f} seconds")
            return value
        return wrapper_timer
    return middle

In [43]:
@wrap_non_picklable_objects
def dot(x,y):
    dot= 0
    for x_i, y_i in zip(x,y):
        dot += x_i * y_i

    return dot

1 process, just a loop

In [44]:
@timer(prefix="Simple python Loop")
def just_looping(X, function: callable):
    n = X.shape[0]
    distance = np.zeros((n,n))
    for i in range(n):
        for j in range(i, n):
            distance[i][j] = function(X[i], X[j])

    return distance

approach1

In [45]:
@wrap_non_picklable_objects
def parallel1(i, j, X, function: callable):
    return function(X[i], X[j])

@timer("Passing entire array")
def test1(X, function, n_jobs=4):
    n = X.shape[0]
    distance = np.zeros((n,n))
    indices = np.argwhere(np.tri(n, M=n, k=-1) == 0)

    distances = np.array(Parallel(n_jobs=n_jobs)(delayed(parallel1)(i,j,X, function) for i,j in indices))
    distance[indices[:,0], indices[:,1]] = distances

    return distance

approach2

In [46]:
@wrap_non_picklable_objects
def parallel2(vec1, vec2, function:callable):
    return function(vec1, vec2)

@timer("Passing individual objects")
def test2(X, function:callable, n_jobs=4):
    n = X.shape[0]
    distance = np.zeros((n,n))
    indices = np.argwhere(np.tri(n, M=n, k=-1) == 0)

    distances = np.array(Parallel(n_jobs=n_jobs)(delayed(parallel2)(X[i], X[j], function) for i,j in indices))
    distance[indices[:,0], indices[:,1]] = distances

    return distance

approach3

In [47]:
from math import ceil

@wrap_non_picklable_objects
def parallel3(indices, X, function):
    return [function(X[i], X[j]) for i,j in indices]

@timer("Splitting equally between n_jobs")
def test3(X, function:callable, n_jobs=4, prefer=None):
    n = X.shape[0]
    distance = np.zeros((n,n))
    indices = np.argwhere(np.tri(n, M=n, k=-1) == 0)
    
    split_size = ceil(len(indices) / n_jobs)
    distances = Parallel(n_jobs=n_jobs, prefer=prefer)(delayed(parallel3)(indices[i*split_size:(i+1)*split_size], X, function) for i in range(n_jobs))

    distance[indices[:,0], indices[:,1]] = np.concatenate(distances)

    return distance


In [50]:
def measure_differences(X, function: callable):
    distances1 = just_looping(X, function)
    distances2 = test1(X, function)
    distances3 = test2(X, function)
    distances4 = test3(X, function)
    print("Are all equal?")
    print(np.array_equal(distances1, distances2), np.array_equal(distances1, distances3), np.array_equal(distances1, distances4))

In [49]:
X = np.random.randn(1000, 100)
measure_differences(X, dot)

Simple python Loop Elapsed time: 11.8424 seconds
Passing entire array Elapsed time: 8.1494 seconds
Passing individual objects Elapsed time: 26.4648 seconds
Splitting equally between n_jobs Elapsed time: 12.7782 seconds
Are all equal?
True True True


In [51]:
measure_differences(X, np.dot)

Simple python Loop Elapsed time: 0.8272 seconds
Passing entire array Elapsed time: 6.5452 seconds
Passing individual objects Elapsed time: 26.3115 seconds
Splitting equally between n_jobs Elapsed time: 1.3416 seconds
Are all equal?
True True True


In [78]:
import networkx as nx
import netrd

X = [nx.Graph(nx.random_regular_graph(10,n=30)) for _ in range(100)]
data = np.empty(len(X), dtype=object)
data[:] = X # Otherwise nx.Graph would be changed into np.array. 
dist = netrd.distance.JaccardDistance()

In [79]:
measure_differences(data, dist)

Simple python Loop Elapsed time: 44.0197 seconds
Passing entire array Elapsed time: 11.9339 seconds
Passing individual objects Elapsed time: 11.1455 seconds
Splitting equally between n_jobs Elapsed time: 10.8027 seconds
Are all equal?
True True True
